In [ ]:
!pip3 uninstall tensorflow
!pip3 install tensorflow-gpu==2.0.0

Uninstalling tensorflow-2.3.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.3.0
     |████████████████████████████████| 380.8MB 39kB/s 
     |████████████████████████████████| 3.8MB 44.9MB/s 
     |████████████████████████████████| 450kB 41.1MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=fa66bb4d94090a4cee0a0f7e9553f4041433b7ca7bb5854bb894678234b847b5
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probabilit

In [ ]:
import os
import re
import pandas as pd
import numpy as np

import tensorflow
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model 
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalMaxPool1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as k

import sklearn
from sklearn import preprocessing
from sklearn.metrics import  accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive
!ls

In [ ]:
df = pd.read_excel('train.xlsx' , encoding = "ISO-8859-1")
df = df[:100000]
df.head(1000)

,kalame,barchasb
0,مدیران,O
1,چهارمین,O
2,جشنواره,O
3,فیلم,O
4,و,O
...,...,...
995,این,O
996,پوشش,O
997,در,O
998,سال,B_DAT


In [ ]:
df.isnull().sum()

kalame      3144
barchasb    3313
dtype: int64

In [ ]:
df = df.fillna(method='ffill')
df.kalame.nunique(), df.barchasb.nunique()

(10750, 15)

In [ ]:
df.groupby('barchasb').size().reset_index(name='counts')

,barchasb,counts
0,B_DAT,579
1,B_LOC,1780
2,B_MON,119
3,B_ORG,2054
4,B_PCT,101
5,B_PER,1551
6,B_TIM,88
7,I_DAT,771
8,I_LOC,760
9,I_MON,279


In [ ]:
#Displaying the unique Barchsb
df['barchasb'].unique()

array(['O', 'B_ORG', 'I_ORG', 'B_PER', 'I_PER', 'B_LOC', 'B_DAT', 'I_DAT',
       'B_TIM', 'I_TIM', 'I_LOC', 'B_MON', 'I_MON', 'B_PCT', 'I_PCT'],
      dtype=object)

In [ ]:
df['kalame'].unique()

array(['مدیران', 'چهارمین', 'جشنواره', ..., 'تذکراتی', 'تقلای',
       'اتهام زنی'], dtype=object)

In [ ]:
n = df['kalame']
n

0         مدیران
1        چهارمین
2        جشنواره
3           فیلم
4              و
          ...   
99995     مواردی
99996       بوده
99997        است
99998         که
99999       دولت
Name: kalame, Length: 100000, dtype: object

In [ ]:
X = df.drop('barchasb', axis=1)
X.head()
X.columns

Index(['kalame'], dtype='object')

In [ ]:
X.shape

(100000, 1)

In [ ]:
df.describe()

,kalame,barchasb
count,100000,100000
unique,10750,15
top,.,O
freq,5251,87089


In [ ]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class jomle(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p) for w, p in zip(s['kalame'].values.tolist(),
                                                 s['barchasb'].values.tolist())]
        self.grouped = self.df.groupby('barchasb').apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['jomle: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [ ]:
#Displaying one full jomle
getter = jomle(df)

In [ ]:
jomle = [' '.join([s[0] for s in sent]) for sent in getter.df]
jomle[0]

'k a l a m e'

In [ ]:
sent = getter.get_text()
print(sent)

None


In [ ]:
#jomle = getter.jomle

In [ ]:
# Number of data points passed in each iteration
batch_size = 64 
# Passes through entire dataset
epochs = 8
# Maximum length of review
max_len = 75 
# Dimension of embedding vector
embedding = 40

In [ ]:
#Getting unique words and labels from data
kalame = list(df['kalame'].unique())
barchasb = list(df['barchasb'].unique())
# Dictionary word:index pair
# word is key and its value is corresponding index
Kalame_to_index = {w : i + 2 for i, w in enumerate(kalame)}
Kalame_to_index["UNK"] = 1
Kalame_to_index["PAD"] = 0

# Dictionary lable:index pair
# label is key and value is index.
barchasb_to_index = {t : i + 1 for i, t in enumerate(barchasb)}
barchasb_to_index["PAD"] = 0

idx2kalame = {i: w for w, i in Kalame_to_index.items()}
idx2barchasb = {i: w for w, i in barchasb_to_index.items()}

In [ ]:
print("The Kalame 'مدیریت ' is identified by the index: {}".format(Kalame_to_index['مدیریت']))
print("The Barchasb B_DAT for the organization is identified by the index: {}".format(barchasb_to_index['B_DAT']))

The Kalame 'مدیریت ' is identified by the index: 605
The Barchasb B_DAT for the organization is identified by the index: 7


In [ ]:
# Converting each sentence into list of index from list of tokens
X = [[kalame_to_index[w[0]] for w in s] for s in jomle]

# Padding each sequence to have same length  of each kalame)
X = pad_sequences(X,maxlen=max_len)

print(X)

NameError: ignored

In [ ]:

# Convert label to index
y = [[barchasb_to_index[w[1]] for w in s] for s in df]

# padding
y = pad_sequences(y,maxlen = max_len)#, jomle = y, padding = 'barchasb', value = barchasb_to_index["PAD"])

IndexError: ignored

In [ ]:
num_barchasb = df['barchasb'].nunique()
# One hot encoded labels
#y = [to_categorical(i, num_classes = num_barchasb + 1) for i in y]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15)

In [ ]:
print("Size of training input data : ", X_train.shape)
print("Size of training output data : ", np.array(y_train).shape)
print("Size of testing input data : ", X_test.shape)
print("Size of testing output data : ", np.array(y_test).shape)

In [ ]:
# Let's check the first sentence before and after processing.
print('*****Before Processing first sentence : *****\n', ' '.join([w[0] for w in sentences[0]]))
print('*****After Processing first sentence : *****\n ', X[0])

In [ ]:
# First label before and after processing.
print('*****Before Processing first sentence : *****\n', ' '.join([w[1] for w in sentences[0]]))
print('*****After Processing first sentence : *****\n ', y[0])

In [ ]:
num_barchasb = df['Tag'].nunique()
# Model architecture
input = Input(shape = (max_len,))
model = Embedding(input_dim = len(Kalame) + 2, output_dim = embedding, input_length = max_len, mask_zero = True)#(input)
model = Bidirectional(LSTM(units = 50, return_sequences=True, recurrent_dropout=0.1))#(model)
model = TimeDistributed(Dense(50, activation="relu"))#(model)
crf = CRF(num_tags+1)  # CRF layer
out = crf(model)  # output

model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

model.summary()